<a id="start"></a>
# Анализ базы данных сервиса чтения книг

### Цели исследования:
* Посчитать, сколько книг вышло после 1 января 2000 года.
* Для каждой книги посчитать количество обзоров и среднюю оценку.
* Определить издательство, которое выпустило наибольшее число книг не считая брошюр.
* Определить автора с самой высокой средней оценкой книг.
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### Содержание:

* [Подготовка данных](#part1)
* [Исследование таблиц](#part2)
* [Запросы](#part3)
    * [Количество вышедших книг](#part3-1)
    * [Количество обзоров и средняя оценка](#part3-2)
    * [Самые активные издательства](#part3-3)
    * [Самые выскооцененный автор](#part3-4)
    * [Количество обзоров от критиков](#part3-5)

## Подготовка данных
<a id="part1"></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas


## Исследование таблиц
<a id="part2"></a>

In [3]:
query = '''SELECT * FROM books LIMIT 5'''

print('books')
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''SELECT * FROM authors LIMIT 5'''

print('authors')
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''

print('publishers')
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''

print('reviews')
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
query = '''SELECT * FROM ratings LIMIT 5'''

print('ratings')
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


## Запросы
<a id="part3"></a>

### Количество вышедших книг
<a id="part3-1"></a>

In [8]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


С первого января 2000 года вышло 819 книг.

### Количество обзоров и средняя оценка
<a id="part3-2"></a>

In [9]:
query = '''
SELECT b.book_id,
       b.title,
       AVG(ra.rating) AS avg_rating,
       COUNT(DISTINCT re.review_id) AS reviews_count
FROM books b
LEFT JOIN reviews re ON b.book_id=re.book_id
LEFT JOIN ratings ra ON b.book_id=ra.book_id
GROUP BY 1
ORDER BY 4 DESC
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,avg_rating,reviews_count
0,948,Twilight (Twilight #1),3.662500,7
1,963,Water for Elephants,3.977273,6
2,734,The Glass Castle,4.206897,6
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
4,695,The Curious Incident of the Dog in the Night-Time,4.081081,6
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0
996,808,The Natural Way to Draw,3.000000,0
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0
998,221,Essential Tales and Poems,4.000000,0


Благодарю за совместную работу и хочу поздравить, ты большой молодец, что прошёл такой путь и освоил программу, надеюсь у тебя сбудутся все намеченные планы.

У Практикума развивается сообщество выпускников, так что рекомендую про него не забывать, в случае необходимости.


Рекомендую всё же написать функцию для первичного анализа, чтобы автоматизировать первичный анализ, один из ценных навыков аналатика - автоматизировать повторяющиеся задачи, в целом есть библиотека - `pandas profiling`, которая как раз позволяет первичный анализ делать ещё легче, может быть ты используешь её). Возможно набор различных фокусов для визуализаций ниже позволит тебе разнообразить твои визуализации и использовать в своей работе больше всего. Желаю успехов!)


Оформление презентаций дело субъективное и зачастую в разных компаниях имеются свои требования, но вот видео от Яндекса по созданию презентаций https://www.youtube.com/watch?v=S0r0fMJa9eA, а ещё на платформе практикума открылся бесплатный курс по созданию презентаций, рекомендую.

Для более глубокого изучения tableau, в случае необходимости рекомендую материал https://tableau.pro/m01, этот курс делает классная команда, у них бывают и другие интересные мини курсы. Но в последнее время российские компании уходят от tableau или уже ушли, поэтому возможно придётся познакомиться с другими инструментами.

Задачи на статистику или вокруг неё часто возникают на собеседовании, для укрепления этого фундамента рекомендую статью для выбора критерия проверки гипотез https://lit-review.ru/biostatistika/vybor-statisticheskogo-kriteriya/, https://statpsy.ru/ - информация по критериям проверки гипотез.

Визуализации данных для аналитиков очень важны, при помощи них легче доносить желаемые мысли, а для того, чтобы лучше отобразить, нужно понимать, какие вообще существуют визуализации, делюсь с тобой своей подборкой для разных случае жизни https://www.notion.so/d9392e1cb71f48cfb1fbf7f557a2b7e7 


SQL важный навык, который нужно поддерживать для этого существует много тренажёров:
* Статья на хабр с топ-тренажёрами: https://techrocks.ru/2019/11/22/7-sites-to-perfect-sql-query-writing-skills/
* задачки SQL - https://habr.com/ru/company/dcmiran/blog/500360/
* https://sqlzoo.net/


In [10]:
query = '''
WITH my_table AS (
SELECT b.book_id,
       b.title,
       AVG(ra.rating) AS avg_rating,
       COUNT(DISTINCT re.review_id) AS reviews_count
FROM books b
LEFT JOIN reviews re ON b.book_id=re.book_id
LEFT JOIN ratings ra ON b.book_id=ra.book_id
GROUP BY 1
ORDER BY 4 DESC)
SELECT SUM(reviews_count)
FROM my_table
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,sum
0,2793.0


In [11]:
query = '''
SELECT COUNT(review_id)
FROM reviews
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


Подготовлена таблица с с средним рейтингом и количеством обзоров для каждой книги.

### Самые активные издательства
<a id="part3-3"></a>

In [12]:
query = '''
SELECT COUNT(b.book_id) AS books_count,
       p.publisher
FROM books b
LEFT JOIN publishers p ON p.publisher_id=b.publisher_id
WHERE num_pages > 50
GROUP BY 2
ORDER BY 1 DESC
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,books_count,publisher
0,42,Penguin Books
1,31,Vintage
2,25,Grand Central Publishing
3,24,Penguin Classics
4,19,Ballantine Books
...,...,...
329,1,Turtleback
330,1,Atheneum Books for Young Readers: Richard Jack...
331,1,Penguin Signet
332,1,Victor Gollancz


Больше всего книг выпустило издательство Penguin Books

### Самые выскооцененный автор
<a id="part3-4"></a>

In [13]:
query = '''
SELECT COUNT(ra.rating_id) AS ratings_count,
       a.author,
       AVG(ra.rating) AS average_rating
FROM books b
LEFT JOIN authors a ON a.author_id=b.author_id
LEFT JOIN ratings ra ON ra.book_id=b.book_id
WHERE b.num_pages > 50
GROUP BY 2
HAVING COUNT(ra.rating_id) > 50
ORDER BY 3 DESC

'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,ratings_count,author,average_rating
0,312,J.K. Rowling/Mary GrandPré,4.288462
1,53,Agatha Christie,4.283019
2,53,Markus Zusak/Cao Xuân Việt Khương,4.264151
3,166,J.R.R. Tolkien,4.240964
4,62,Roald Dahl/Quentin Blake,4.209677
5,54,Louisa May Alcott,4.203704
6,84,Rick Riordan,4.130952
7,56,Arthur Golden,4.107143
8,106,Stephen King,4.009434
9,70,John Grisham,3.971429


Писательница Джоан Роулинг и иллюстратор ее книг Мэри Гранпрэ на первом месте по средней оценке ее книг.

### Количество обзоров от критиков
<a id="part3-5"></a>

In [14]:
query = '''
WITH critics_reviews AS (
SELECT COUNT(re.review_id) AS review_count,
       re.username
FROM reviews re
GROUP BY 2
ORDER BY 1 DESC),
critics_ratings AS (
SELECT COUNT(ra.rating_id) AS rating_count,
       ra.username
FROM ratings ra
GROUP BY 2
ORDER BY 1 DESC
)
SELECT AVG(cre.review_count)
FROM critics_reviews cre
JOIN critics_ratings cra ON cre.username=cra.username
WHERE cra.rating_count > 48

'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, равняется 24 обзорам.

[В начало](#start)